In [10]:
import pandas as pd
import os
import csv
import json
import math

In [ ]:
# INSTRUCTIONS (Download the Reactome Neo4j graph):
# Download the Reactome database as a Neo4j graph: https://reactome.org/download-data/ is the main page
# These are the possible files: https://reactome.org/download/current/reactome.graphdb.tgz 
# or https://reactome.org/download/current/reactome.graphdb.dump\n",
Neo4j Command:
    MATCH p=(pw:Pathway)-[edge:hasEvent]-(r:ReactionLikeEvent)
    WHERE toLower(pw.speciesName) = 'homo sapiens'
    WITH collect(pw) as pathways, collect(r) as RLEs , collect(edge) as relation
    CALL apoc.export.csv.data(pathways + RLEs, relation, 'human_pathway_reaction.csv', {})
    YIELD file, source, format, nodes, relationships, properties, time, rows, batchSize, batches, done, data
    RETURN file, source, format, nodes, relationships, properties, time, rows, batchSize, batches, done, data

In [2]:
nodes = pd.read_csv('input/human_pathway_reaction.csv')[['_id', 'stId','displayName', 'name']]
edges = pd.read_excel('input/human_pathways_reaction-like-event_edges.xlsx')

/home/ubuntu/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (1,2,4,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [12]:
db2reactome_id = dict(zip(nodes._id, nodes.stId))
db2reactome_id = {int(db):reactome_id for db, reactome_id in db2reactome_id.items() if not math.isnan(db)}
#db2reactome_id[931828]
edges.tail()

,_start,_end,_type,order
14920,1474231,1475330,hasEvent,1
14921,1697874,1698341,hasEvent,7
14922,1864535,1864536,hasEvent,1
14923,729530,730703,hasEvent,9
14924,1609237,1609250,hasEvent,28


In [13]:
# Edges
edge_list = list()

for i in range(0,len(edges)):
    pathway = edges['_start'].iloc[i]
    reaction = edges['_end'].iloc[i]
    rel = edges['_type'].iloc[i]
    
    try:
        pathway = db2reactome_id[pathway]
        reaction = db2reactome_id[reaction]        
        edge_list.append([pathway, reaction])
        
    except:
        continue

In [16]:
# Output Edges
file = 'Reaction_(Reactome)_2_Pathway_(Reactome).csv'
with open(os.path.join('output/pathway2reaction',file),'w') as fout:
    writer = csv.writer(fout)
    writer.writerow(['Reaction (Reactome)','Pathway (Reactome)','Relationship'])
    relationship = '-involved_in->'
            

    for edge in edge_list:
        pathway = edge[0]
        reaction = edge[1]
        writer.writerow(['Reactome_Reaction:'+reaction, 'Reactome_Pathway:'+pathway, relationship])

In [17]:
df = pd.read_csv(os.path.join('output/pathway2reaction',file))
df.to_csv(os.path.join('output/edges/', file), index=False)
df.to_csv(os.path.join('output/edges to use/',file), index=False)
df

,Reaction (Reactome),Pathway (Reactome),Relationship
0,Reactome_Reaction:R-HSA-8851908,Reactome_Pathway:R-HSA-8851907,-involved_in->
1,Reactome_Reaction:R-HSA-2265534,Reactome_Pathway:R-HSA-2206308,-involved_in->
2,Reactome_Reaction:R-HSA-8852019,Reactome_Pathway:R-HSA-8851907,-involved_in->
3,Reactome_Reaction:R-HSA-8865774,Reactome_Pathway:R-HSA-8955332,-involved_in->
4,Reactome_Reaction:R-HSA-9697747,Reactome_Pathway:R-HSA-5357905,-involved_in->
...,...,...,...
14920,Reactome_Reaction:R-HSA-3301237,Reactome_Pathway:R-HSA-3214847,-involved_in->
14921,Reactome_Reaction:R-HSA-9611852,Reactome_Pathway:R-HSA-390696,-involved_in->
14922,Reactome_Reaction:R-HSA-2328129,Reactome_Pathway:R-HSA-3000171,-involved_in->
14923,Reactome_Reaction:R-HSA-8850945,Reactome_Pathway:R-HSA-8948751,-involved_in->


In [27]:
len(set(df['Pathway (Reactome)']))

2127

In [28]:
len(set(df['Reaction (Reactome)']))

14094

#### 14925 